In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

!pip install gurobipy>=9.5.1
import gurobipy as gp
from gurobipy import GRB as GRB
import numpy as np

In [ ]:
# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '7e2d40a7-904b-4d00-b37c-6993c3716fb6')
e.setParam('WLSSECRET', '731bbd0f-37ee-4c88-9d28-c4f67b9c7952')
e.setParam('LICENSEID', 2396892)
e.start()

# Create the model within the Gurobi environment
model = gp.Model(env=e)
model2 = gp.Model(env=e)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2396892
Academic license - for non-commercial use only - registered to tenns@uwaterloo.ca


**Expanded Model with DeVinci Data**

In [ ]:
#Define sets & Params
N = ['X', 'W', 'Y', 'T','G','S','T_s','S_s','O','F','P','H','P_s','Q','A','B','C']
K = [1,2,3,4,5,6]
r = {
    'A': 1000,
    'B': 700,
    'C': 3400,
    'H': 200,
    'T': 700,
    'T_s': 400,
    'G': 300,
    'Q': 0,
    'P': 2000,
    'P_s': 400,
    'S': 75,
    'S_s': 50,
    'O': 100,
    'F': 20,
    'W': 25,
    'X': 75,
    'Y': 10
}
v = {
    'A': 50,
    'B': 50,
    'C': 50,
    'H': 5,
    'T': 20,
    'T_s': 15,
    'G': 10,
    'Q': 0,
    'P': 30,
    'P_s': 10,
    'S': 3,
    'S_s': 2,
    'O': 5,
    'F': 0,
    'W': 1,
    'X': 3,
    'Y': 0
}
c = {
    'A': 800,
    'B': 500,
    'C': 2800,
    'H': 50,
    'T': 400,
    'T_s': 200,
    'G': 100,
    'Q': 80,
    'P': 1600,
    'P_s': 200,
    'S': 40,
    'S_s': 25,
    'O': 50,
    'F': 5,
    'W': 15,
    'X': 20,
    'Y': 5
}
b = {1:10,2:20,3:30,4:15,5:15,6:10}
p = {
    'X':  {'X':1, 'W':0, 'Y':0, 'T':0, 'G':0, 'S':0, 'T_s':0, 'S_s':0, 'O':0, 'F':0, 'P':0, 'H':0, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'W':  {'X':0, 'W':1, 'Y':0, 'T':0, 'G':0, 'S':0, 'T_s':0, 'S_s':0, 'O':0, 'F':0, 'P':0, 'H':0, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'Y':  {'X':0, 'W':0, 'Y':1, 'T':0, 'G':0, 'S':0, 'T_s':0, 'S_s':0, 'O':0, 'F':0, 'P':0, 'H':0, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'T':  {'X':0, 'W':0, 'Y':0, 'T':1, 'G':0, 'S':0, 'T_s':0, 'S_s':0, 'O':0, 'F':0, 'P':0, 'H':0, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'G':  {'X':-1,'W':-3,'Y':-2,'T':0, 'G':1, 'S':0, 'T_s':0, 'S_s':0, 'O':0, 'F':0, 'P':0, 'H':0, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'S':  {'X':0, 'W':0, 'Y':0, 'T':0, 'G':0, 'S':1, 'T_s':0, 'S_s':0, 'O':0, 'F':0, 'P':0, 'H':0, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'T_s':{'X':0, 'W':0, 'Y':0, 'T':0, 'G':0, 'S':0, 'T_s':1,'S_s':0, 'O':0, 'F':0, 'P':0, 'H':0, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'S_s':{'X':0, 'W':0, 'Y':0, 'T':0, 'G':0, 'S':0, 'T_s':0, 'S_s':1,'O':0, 'F':0, 'P':0, 'H':0, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'O':  {'X':0, 'W':0, 'Y':0, 'T':0, 'G':0, 'S':0, 'T_s':0, 'S_s':0, 'O':1, 'F':0, 'P':0, 'H':0, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'F':  {'X':0, 'W':0, 'Y':0, 'T':0, 'G':0, 'S':0, 'T_s':0, 'S_s':0, 'O':0, 'F':1, 'P':0, 'H':0, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'P':  {'X':0, 'W':0, 'Y':0, 'T':-1,'G':-1,'S':-1,'T_s':0, 'S_s':0, 'O':0, 'F':0, 'P':1, 'H':0, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'H':  {'X':0, 'W':0, 'Y':0, 'T':0, 'G':0, 'S':0, 'T_s':0, 'S_s':0, 'O':0, 'F':0, 'P':0, 'H':1, 'P_s':0, 'Q':0, 'A':0, 'B':0, 'C':0},
    'P_s':{'X':0, 'W':0, 'Y':0, 'T':0, 'G':-1,'S':0, 'T_s':-1,'S_s':-1,'O':0, 'F':0, 'P':0, 'H':0, 'P_s':1,'Q':0, 'A':0, 'B':0, 'C':0},
    'Q':  {'X':0, 'W':0, 'Y':0, 'T':0, 'G':0, 'S':0, 'T_s':0, 'S_s':0, 'O':-1,'F':-1,'P':0, 'H':0, 'P_s':0, 'Q':1, 'A':0, 'B':0, 'C':0},
    'A':  {'X':0, 'W':0, 'Y':0, 'T':-1,'G':-1,'S':0, 'T_s':0, 'S_s':0, 'O':0, 'F':0, 'P':0,'H':-1, 'P_s':0,'Q':-2,'A':1, 'B':0, 'C':0},
    'B':  {'X':0, 'W':0, 'Y':0, 'T':0,'G':0,'S':0, 'T_s':0, 'S_s':0, 'O':-2, 'F':0, 'P':0,'H':-1,'P_s':-1,'Q':0,'A':0, 'B':1, 'C':0},
    'C':  {'X':0, 'W':0, 'Y':0, 'T':0,'G':0,'S':0,'T_s':0,'S_s':0,'O':-2,'F':0,'P':-1,'H':-1,'P_s':0,'Q':0,'A':0,'B':0,'C':1},
}

a = {
    'X': {1: 0.05, 2: 0.2, 3: 0, 4: 0.05, 5: 0, 6: 0},
    'W': {1: 0.05, 2: 0.3, 3: 0, 4: 0.1, 5: 0, 6: 0},
    'Y': {1: 0.05, 2: 0.05, 3: 0, 4: 0.02, 5: 0, 6: 0},
    'T': {1: 0, 2: 0.3, 3: 0.4, 4: 0.8, 5: 2, 6: 0},
    'G': {1: 0, 2: 0.6, 3: 0, 4: 0.7, 5: 0.2, 6: 0},
    'S': {1: 0, 2: 0.2, 3: 0, 4: 0.7, 5: 1, 6: 0},
    'T_s': {1: 0, 2: 0.2, 3: 0.3, 4: 0.7, 5: 1.5, 6: 0},
    'S_s': {1: 0, 2: 0.15, 3: 0, 4: 0.65, 5: 1, 6: 0},
    'O': {1: 0, 2: 0.15, 3: 1, 4: 0.8, 5: 0.3, 6: 0},
    'F': {1: 0, 2: 0.1, 3: 0, 4: 0.1, 5: 0.1, 6: 0},
    'P': {1: 0, 2: 1, 3: 0.5, 4: 1, 5: 2, 6: 0},
    'H': {1: 0, 2: 0.2, 3: 0.5, 4: 0.5, 5: 0, 6: 0},
    'P_s': {1: 0, 2: 1, 3: 0.5, 4: 0.8, 5: 1.7, 6: 0},
    'Q': {1: 0, 2: 0.4, 3: 1, 4: 1, 5: 0.5, 6: 0},
    'A': {1: 0.5, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0.3},
    'B': {1: 0.5, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0.3},
    'C': {1: 0.3, 2: 1, 3: 0.8, 4: 1, 5: 1, 6: 0.2},
}

g = {1:4000,2:18000,3:17000,4:19000,5:19000,6:2000}
h = {1:0.25,2:0.25,3:0.25,4:0.25,5:0.25,6:0.25}
u = {
    'A': 4796,
    'B': 1996,
    'C': 3196,
    'H': 217,
    'T': 477,
    'T_s': 58,
    'G': 16,
    'Q': 0,
    'P': 436,
    'P_s': 97,
    'S': 357,
    'S_s': 25,
    'O': 636,
    'F': 436,
    'W': 157,
    'X': 138,
    'Y': 556
}
#For second period of year
# u = {
#     'A': 5998,
#     'B': 2498,
#     'C': 3998,
#     'H': 257,
#     'T': 537,
#     'T_s': 86,
#     'G': 20,
#     'Q': 0,
#     'P': 518,
#     'P_s': 137,
#     'S': 417,
#     'S_s': 41,
#     'O': 738,
#     'F': 518,
#     'W': 197,
#     'X': 156,
#     'Y': 758
# }
l = {
    'A': 1000,
    'B': 1000,
    'C': 1000,
    'H': 40,
    'T': 50,
    'T_s': 10,
    'G': 2,
    'Q': 0,
    'P': 50,
    'P_s': 87,
    'S': 70,
    'S_s': 5,
    'O': 120,
    'F': 60,
    'W': 20,
    'X': 20,
    'Y': 20
}

In [ ]:
#Decision Variables
X = model.addVars(N, vtype=GRB.INTEGER, name="X")
Y = model.addVars(N, vtype=GRB.INTEGER, name="Y")
E = model.addVars(K, vtype=GRB.CONTINUOUS, name="E")
F = model.addVars(K, vtype=GRB.CONTINUOUS, name="F")

#Constraints
[model.addConstr(X[j] == sum(Y[i]*p[i][j] for i in N)) for j in N]
[model.addConstr(sum(Y[i]*a[i][k] for i in N) <= E[k]+F[k]) for k in K]
[model.addConstr(E[k] <= g[k]) for k in K]
[model.addConstr(F[k] <= h[k]*g[k]) for k in K]
[model.addConstr(X[i] <= u[i]) for i in N]
[model.addConstr(X[i] >= l[i]) for i in N]
[model.addConstr(X[i] >= 0) for i in N]
[model.addConstr(Y[i] >= 0) for i in N]
[model.addConstr(E[k] >= 0) for k in K]
[model.addConstr(F[k] >= 0) for k in K]

#Set Objective
model.setObjective(sum((r[i]-c[i])*X[i] for i in N) - sum(F[k]*b[k] for k in K), sense=GRB.MAXIMIZE)

model.optimize()

model.printAttr('X')


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license - for non-commercial use only - registered to tenns@uwaterloo.ca
Optimize a model with 115 rows, 46 columns and 225 nonzeros
Model fingerprint: 0x82a2d657
Variable types: 12 continuous, 34 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 3e+00]
  Objective range  [5e+00, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 104 rows and 19 columns
Presolve time: 0.00s
Presolved: 11 rows, 27 columns, 85 nonzeros
Variable types: 0 continuous, 27 integer (0 binary)
Found heuristic solution: objective 1043172.5500
Found heuristic solution: objective 1060487.0500

Root relaxation: objective 1.366001e+06, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds  

**Second Half of the Year**


In [ ]:
# For second period of year
u = {
    'A': 5998,
    'B': 2498,
    'C': 3998,
    'H': 257,
    'T': 537,
    'T_s': 86,
    'G': 20,
    'Q': 0,
    'P': 518,
    'P_s': 137,
    'S': 417,
    'S_s': 41,
    'O': 738,
    'F': 518,
    'W': 197,
    'X': 156,
    'Y': 758
}

#Decision Variables
X = model2.addVars(N, vtype=GRB.INTEGER, name="X")
Y = model2.addVars(N, vtype=GRB.INTEGER, name="Y")
E = model2.addVars(K, vtype=GRB.CONTINUOUS, name="E")
F = model2.addVars(K, vtype=GRB.CONTINUOUS, name="F")

#Constraints
[model2.addConstr(X[j] == sum(Y[i]*p[i][j] for i in N)) for j in N]
[model2.addConstr(sum(Y[i]*a[i][k] for i in N) <= E[k]+F[k]) for k in K]
[model2.addConstr(E[k] <= g[k]) for k in K]
[model2.addConstr(F[k] <= h[k]*g[k]) for k in K]
[model2.addConstr(X[i] <= u[i]) for i in N]
[model2.addConstr(X[i] >= l[i]) for i in N]
[model2.addConstr(X[i] >= 0) for i in N]
[model2.addConstr(Y[i] >= 0) for i in N]
[model2.addConstr(E[k] >= 0) for k in K]
[model2.addConstr(F[k] >= 0) for k in K]

#Set Objective
model2.setObjective(sum((r[i]-c[i])*X[i] for i in N) - sum(F[k]*b[k] for k in K), sense=GRB.MAXIMIZE)

model2.optimize()

model2.printAttr('X')


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license - for non-commercial use only - registered to tenns@uwaterloo.ca
Optimize a model with 115 rows, 46 columns and 225 nonzeros
Model fingerprint: 0x6a5386cd
Variable types: 12 continuous, 34 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 3e+00]
  Objective range  [5e+00, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 104 rows and 19 columns
Presolve time: 0.00s
Presolved: 11 rows, 27 columns, 85 nonzeros
Variable types: 0 continuous, 27 integer (0 binary)
Found heuristic solution: objective 1043172.5500
Found heuristic solution: objective 1055456.2000
Found heuristic solution: objective 1062159.4500

Root relaxation: objective 1.393835e+06, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes

In [ ]:
!jupyter nbconvert 434model_DeVinci.ipynb --to html

[NbConvertApp] Converting notebook 434model_DeVinci.ipynb to html
[NbConvertApp] Writing 663307 bytes to 434model_DeVinci.html
